In [1]:
import glob, os
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

df = pd.read_csv('example.csv')
df1 = pd.read_csv('elo.csv')

In [2]:
df['elo_home'] = df1['elo_home']
df['elo_away'] = df1['elo_away']
df['score_gap'] = df1['score_gap']
df['results'] = df1['results']

In [3]:
def cal_score(row):
    if row['results'] == 1:
        return row['score_gap']
    else:
        return (-1)*row['score_gap']
df['score_gap_sign'] = df.apply(cal_score, axis = 1)

In [4]:
df['elo_diff'] = df['elo_home'] - df['elo_away']

In [5]:
df.head()

,Date,Start (ET),Visitor/Neutral,Visitor-Points,Home/Neutral,Home-Points,Overtime,Attend.,Season,Age_Visitor,...,T_BLK_Home,T_TOV_Home,T_PF_Home,T_PTS_Home,elo_home,elo_away,score_gap,results,score_gap_sign,elo_diff
0,Tue Oct 29 2013,7:00 pm,Orlando Magic,87,Indiana Pacers,97,0,18165,2013/2014,24.6,...,5.4,15.1,20.4,96.7,1509.424989,1490.575011,10,1,10,18.849977
1,Tue Oct 29 2013,10:30 pm,Los Angeles Clippers,103,Los Angeles Lakers,116,0,18997,2013/2014,28.1,...,5.4,15.1,19.8,103.0,1509.424989,1490.575011,13,1,13,18.849977
2,Tue Oct 29 2013,8:00 pm,Chicago Bulls,95,Miami Heat,107,0,19964,2013/2014,28.6,...,4.5,14.8,19.5,102.2,1509.424989,1490.575011,12,1,12,18.849977
3,Wed Oct 30 2013,7:00 pm,Brooklyn Nets,94,Cleveland Cavaliers,98,0,20562,2013/2014,30.0,...,3.7,14.2,20.0,98.2,1509.424989,1490.575011,4,1,4,18.849977
4,Wed Oct 30 2013,8:30 pm,Atlanta Hawks,109,Dallas Mavericks,118,0,19834,2013/2014,27.5,...,4.3,13.5,20.0,104.8,1509.424989,1490.575011,9,1,9,18.849977


In [6]:
df = df.drop(['results', 'score_gap', 'Date', 'Start (ET)', 
              'Visitor/Neutral', 'Home/Neutral', 'Attend.', 'Season', 
             'O_G_Visitor', 'O_G_Home', 'T_G_Visitor', 'T_G_Home', 
             'elo_home', 'elo_away'], 1)
df.shape

(6567, 137)

In [7]:
df['new_score_gap_sign'] = df['score_gap_sign']
df = df.drop(['score_gap_sign'], 1)
df.rename(columns={'new_score_gap_sign': 'score_gap_sign'}, inplace=True)

In [8]:
df_clean = df.iloc[600:]
df_clean = df_clean.reset_index(drop = True)

In [9]:
X = df_clean.drop(['score_gap_sign'], axis = 1)
y = df_clean['score_gap_sign']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 66)

In [10]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [11]:
pred = linreg.predict(X_test)
error = np.sqrt(mean_squared_error(y_test, pred))
print(error)

8.149272846908313e-14


In [12]:
df_clean.to_csv('final.csv', index=False)

In [15]:
out_of_sample = pd.read_csv('Out_of_Sample.csv')
out_of_sample = out_of_sample.drop(['Visitor-Points', 'Home-Points'], 1)
X = X.drop(['Visitor-Points', 'Home-Points'], 1)

In [16]:
# out-of-sample prediction
X_out = out_of_sample.drop(['score_gap_sign'], axis = 1)
y_out = out_of_sample['score_gap_sign']
linreg.fit(X,y)
pred = linreg.predict(X_out)
error = np.sqrt(mean_squared_error(y_out, pred))
print(pred)
print(error)

[ 4.36044822e+00  2.20826252e+01 -4.52961451e+00 -5.32154543e+00
 -1.38928760e+01  1.80063134e+01 -2.28286409e+00  1.72029863e+00
 -1.34222999e+01 -9.39860526e+00 -7.04605480e+00  2.63887415e+00
  1.90218330e+01  9.85870027e+00 -2.63845811e-01 -1.63678795e+01
 -3.15572781e+00  2.16473582e+00  9.81621458e+00  7.91847220e+00
 -7.11302796e+00  1.49088041e+01 -1.53058262e+01  9.48845905e+00
 -1.09402674e+01 -4.00417204e+00 -2.70366562e+01  3.23456544e+01
  1.56187705e+01  8.72574168e-01 -2.71274764e+00 -1.52355008e+01
  8.62741204e+00  5.82849792e-01 -3.59768747e+01 -4.11002245e+00
  2.46538344e-02  3.68544276e+00  6.20053753e+00  1.35364127e+01
 -1.53056137e+01  4.26731113e+01 -2.76620060e+00  2.90193222e+01
 -2.32512773e+01  4.79302236e+00  1.50679452e+01  8.93144045e+00
  1.97953582e+01 -1.42181306e+01  6.15731424e+00 -9.18822339e+00
 -1.17207565e+01 -1.27465161e+01  2.75839345e+01 -1.42367766e+01
  7.30957727e+00  1.54420562e+01 -3.00241744e+01  5.93585563e+00
 -5.75629601e+00  1.13873